In [12]:
import pandas as pd
import geopy.distance

In [4]:
df = pd.read_csv("201809-citibike-tripdata.csv")
pd.set_option('display.max_rows', None)

### 13.5.3 Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов

In [6]:
df.shape

(1877884, 15)

### 13.5.4 Найти среднюю длину поездок в минутах(столбец tripduration) c точностью до 2 знака

In [10]:
round(df["tripduration"].mean()/60,2)

16.13

### 13.5.5 Сколько поездок начались и закончились в той же самой станции?

In [15]:
df[df["start station name"]==df["end station name"]].shape[0]

41364

### 13.5.6 Какой самый используемый байк(bikeid) в городе по количеству поездок?

In [86]:
df.groupby(["bikeid"], as_index=False).count().sort_values("tripduration", ascending=False)["bikeid"].head(n=1)

10105    33875
Name: bikeid, dtype: int64

### 13.5.7 Найдите идентификатор велосипеда (bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных

In [112]:
df.groupby(["bikeid"], as_index=False).mean().sort_values("tripduration", ascending=False).head(n=1)["bikeid"]

1840    17548
Name: bikeid, dtype: int64

### 13.5.8 Сколько строк, в которых отсутствуют данные о start station id?

In [102]:
df[df["start station id"].isnull()]["bikeid"].count()

716

### 13.5.9 Какова средняя продолжительность поездки в минутах в зависимости от типа подписки c точностью до 2 знака?

In [111]:
result = df.groupby(["usertype"], as_index=False).mean()[["usertype", "tripduration"]]
result["tripduration"] = round(result["tripduration"]/60, 2)
result

,usertype,tripduration
0,Customer,33.42
1,Subscriber,13.33


### 13.5.10 Найдите среднюю длину поездок в километрах с точностью до 2 знака, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id).

Hint: можно воспользоваться библиотекой geopy и взять расстояние vincenty(минимальное расстояние между точками)

In [18]:
trips = df[df["start station id"] != df["end station id"]]

In [29]:
def get_distance(row):
    start_point = (row["start station latitude"], row["start station longitude"])
    finish_point = (row["end station latitude"], row["end station longitude"])
    return geopy.distance.distance(start_point, finish_point).km

round(trips.apply(get_distance, axis=1).mean(),2) #very slow solution

1.85

### 13.5.11 Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера включительно

In [30]:
# convert columns to time
# filter by time range
# group by station id
# get max value



,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877879,369,2018-09-30 00:27:25.9840,2018-09-30 00:33:35.7070,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3631.0,Crown St & Bedford Ave,40.666563,-73.956741,32976,Subscriber,1985,2
1877880,191,2018-09-30 00:30:30.1850,2018-09-30 00:33:42.1090,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3631.0,Crown St & Bedford Ave,40.666563,-73.956741,15595,Subscriber,1985,2
1877881,1442,2018-09-30 08:10:03.1790,2018-09-30 08:34:05.3870,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,471.0,Grand St & Havemeyer St,40.712868,-73.956981,28646,Subscriber,1981,1
1877882,453,2018-09-30 12:20:13.6830,2018-09-30 12:27:46.9140,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3584.0,Eastern Pkwy & Franklin Ave,40.670777,-73.957680,34272,Subscriber,1986,1
